In [2]:
import os
import numpy as np
import plyfile

def read_mesh(filename):
    """Reads a PLY mesh file and extracts vertex and face information.
    
    Args:
        filename (str): Path to the PLY file.
    
    Returns:
        tuple: A tuple containing vertices and faces.
    """
    assert os.path.isfile(filename)
    
    # Read PLY data from file
    with open(filename, 'rb') as f:
        plydata = plyfile.PlyData.read(f)
        num_verts = plydata['vertex'].count
        
        # Initialize an array to store vertex information
        vertices = np.zeros(shape=[num_verts, 8], dtype=np.float32)
        
        # Populate the vertex array with data from PLY file
        vertices[:, 0] = plydata['vertex'].data['x']
        vertices[:, 1] = plydata['vertex'].data['y']
        vertices[:, 2] = plydata['vertex'].data['z']
        vertices[:, 3] = plydata['vertex'].data['red']
        vertices[:, 4] = plydata['vertex'].data['green']
        vertices[:, 5] = plydata['vertex'].data['blue']
        vertices[:, 6] = plydata['vertex'].data['alpha']
        vertices[:, 7] = plydata['vertex'].data['label']

    # Return vertices and faces
    return vertices, plydata['face']

def write_mesh(vertices, faces):
    """Writes vertices and faces to a new PLY mesh file.
    
    Args:
        vertices (numpy.ndarray): Array containing vertex information.
        faces (numpy.ndarray): Array containing face information.
    """
    new_vertices = []
    
    # Prepare vertices array for writing
    for i in range(vertices.shape[0]):
        new_vertices.append(
            (
                vertices[i][0],
                vertices[i][1],
                vertices[i][2],
                vertices[i][3],
                vertices[i][4],
                vertices[i][5],
                vertices[i][6],
                vertices[i][7],
            )
        )

    # Create structured array for vertices
    vertices = np.array(
        new_vertices,
        dtype=[
            ("x", np.dtype("float32")),
            ("y", np.dtype("float32")),
            ("z", np.dtype("float32")),
            ("red", np.dtype("uint8")),
            ("green", np.dtype("uint8")),
            ("blue", np.dtype("uint8")),
            ("alpha", np.dtype("uint8")),
            ("label", np.dtype("ushort"))
        ]
    )

    # Create PlyElement and PlyData for writing
    vertices = plyfile.PlyElement.describe(vertices, "vertex")
    mesh = plyfile.PlyData([vertices, faces])
    
    # Write the mesh to a new PLY file
    mesh.write("./scene0012_00_aligned_mesh2.ply")



In [4]:
import os
import plyfile
import numpy as np

# Define the path to the PLY file
filename = os.path.join("D:/aaaaaaaaaaaaaaaaa/23-24/astar/1-2data/ScanNet/scannet/scans/scene0012_00/scene0012_00_vh_clean_2.labels.ply")

# Read vertices and faces from the PLY file
vertices, faces = read_mesh(filename)

# Define the path to the metadata file
meta_file = os.path.join("D:/aaaaaaaaaaaaaaaaa/23-24/astar/1-2data/ScanNet/scannet/scans/scene0012_00/scene0012_00.txt")

# Read lines from the metadata file
lines = open(meta_file).readlines()

# Initialize the variable for storing the axis alignment matrix
axis_align_matrix = None

# Find the line containing 'axisAlignment' and extract the matrix values
for line in lines:
    if 'axisAlignment' in line:
        axis_align_matrix = [float(x) for x in line.rstrip().strip('axisAlignment = ').split(' ')]

# Check if the axis alignment matrix was found
if axis_align_matrix is not None:
    # Reshape the matrix into a 4x4 array
    axis_align_matrix = np.array(axis_align_matrix).reshape((4, 4))
    
    # Create homogeneous coordinates for vertices and transform them using the matrix
    pts = np.ones((vertices.shape[0], 4))
    pts[:, 0:3] = vertices[:, :3]
    pts = np.dot(pts, axis_align_matrix.transpose())
    
    # Copy the original vertices and update their positions with aligned coordinates
    aligned_vertices = np.copy(vertices)
    aligned_vertices[:, 0:3] = pts[:, 0:3]

    # Write the aligned vertices and faces to a new PLY file
    write_mesh(aligned_vertices, faces)

